In [7]:
import requests
import json
from datetime import date
from time import sleep
from pathlib import Path
import polars as pl

base = Path.home() / "Local_Workspace" / "Datasets" / "ARPA" / "MARCHE" / "ASSAM"

In [2]:
def request_data(province, date: date):
  return {
    "Utente": "9",
    "SiglaProv": province,
    "Giorno": date.strftime(r"%d/%m/%Y")
  }
headers = {
  'Content-Type': 'application/json'
}

provinces = ["AN", "AP", "FM", "MC", "PU"]

In [10]:
def download_date(date: date, session: requests.Session):
  path = base / "fragments" / f"{date.strftime(r"%d-%m-%Y")}.csv"
  path.parent.mkdir(exist_ok=True)
  resps = []
  for province in provinces:
    data = request_data(province, date)
    resp = session.post("http://meteo.marche.it/ws2009/DISPENSERservice.asmx/MisureGiornalierePerProvincia", json=data).json()
    for i in range(len(resp)):
      resp[i]["province_code"] = province
    resps.extend(resp)
    sleep(0.2)
  pl.from_dicts(resps).write_csv(path)



In [14]:
with requests.Session() as session:
    download_date(date(1994, 12, 29), session)